In [ ]:
!pip install easyocr

In [5]:
# Importing necessary modules
import os
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import easyocr

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
def get_bigger_box(res):
    biggerBox = []
    area = 0
    minY = 1000
    for i in res:
        box = []
        box.append(i[0][0])
        box.append(i[0][2])
        print(i)
        box = [item for sublist in box for item in sublist]
        box = [int(item) for item in box]
        textArea = abs((box[0] - box[1]) * (box[2] - box[3]))
        if textArea > area and box[1] < minY:
            minY = box[1]
            area = textArea
            biggerBox = box
    return biggerBox

In [7]:
def crop_lettering(final, biggerText):
    top_left = (biggerText[0], biggerText[1])
    bottom_right = (biggerText[2], biggerText[3])
    imag = final[top_left[1] : bottom_right[1], top_left[0] : bottom_right[0]]
    return imag

In [8]:
def readText(filename):
    # Using EasyOCR to detect text boxes
    reader = easyocr.Reader(['pt', 'en'], recognizer='Transformer')
    res = reader.readtext(filename)
    return res

In [9]:
def rec(filenameOpen, valterBox):
    imag = cv2.imread(filenameOpen)
    imag = cv2.rectangle(imag, (valterBox[0], valterBox[1]), (valterBox[2], valterBox[3]), (0, 255, 0))
    return imag

In [3]:
# Returns the similarity between two strings
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
# Google Drive path to dataset
path = ""



for i in range(6327):
    if not os.path.isfile(f"{path}img{i}.jpg"):
        continue
    image = cv2.imread(f"{path}img{i}.jpg")
    image = cv2.GaussianBlur(image, (5,5), 0.5)
    biggerText = get_bigger_box(readText(image))
    biggerText[0] -= 10
    biggerText[1] -= 10
    biggerText[2] += 10
    biggerText[3] += 10
    if biggerText[0] < 0 or biggerText[1] < 0:
        continue
    lettering = crop_lettering(image, biggerText)
    res = readText(255-cv2.cvtColor(lettering, cv2.COLOR_BGR2RGB))
    plt.imshow(255-cv2.cvtColor(lettering, cv2.COLOR_BGR2RGB))
    plt.show()
    print(res[0][1])